In [63]:
from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector
import random
from mesa.visualization.ModularVisualization import ModularServer
from mesa.visualization.modules import CanvasGrid, ChartModule, TextElement
from mesa.visualization.UserParam import UserSettableParameter

class CovAgent(Agent):

    def __init__(self, pos, model, agent_type,port_mask,immune,prob_inf=None,temps_inf=None):
        #agent_type: Indicator for the agent's type (suceptible=0, infecte=1,retabli=-1)
        super().__init__(pos, model)
        self.pos = pos
        self.type = agent_type
        self.port_mask = port_mask
        self.immune = immune
        self.prob_inf = prob_inf
        self.temps_inf = temps_inf
        self.deb_inf=0

    def step(self):
        self.sucep2infect()
        self.infect2retab()
        if self.type== 1:
            self.model.infecte+=1
        if self.type== 0:
            self.model.sucep+=1
        if self.type== -1:
            self.model.retab+=1
        if not self.model.confinement:
            self.model.grid.move_to_empty(self)

            
            
     
    def sucep2infect(self):
        prob_inff=self.prob_inf
        if self.type==1 :
            for neighbor in self.model.grid.neighbor_iter(self.pos):
                if neighbor.type==0:
                    if self.port_mask==1 and neighbor.port_mask==1 :
                        prob_inff-=prob_inff*0.985
                    if self.port_mask==1 and neighbor.port_mask==0 :
                        prob_inff-=prob_inff*0.95
                    if self.port_mask==0 and neighbor.port_mask==1 :
                        prob_inff-=prob_inff*0.3
                    if prob_inff>neighbor.immune:
                        neighbor.type=1 
                        neighbor.immune=None
                        neighbor.deb_inf=self.model.runs
                        neighbor.prob_inf=random.uniform(0,1)
                        neighbor.temps_inf=random.randint(5,15)
    def infect2retab(self):
        if self.type==1:
            temp=(self.model.runs-self.deb_inf)
            if temp>self.temps_inf:
                self.type=-1
                

                

In [64]:
class CovModel(Model):

    def __init__(self, height=5, width=5, density=0.8, minority_pc=0.2,d_pm=0.8,confinement=False):
        self.height = height
        self.width = width
        self.density = density
        self.minority_pc = minority_pc
        self.d_pm = d_pm
        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(width, height, torus=True)
        self.confinement=confinement
        self.infecte = 0
        self.sucep=0
        self.retab=0
        self.runs=0
        self.datacollector = DataCollector(model_reporters={"infecte": "infecte","suceptible": "sucep","retabli": "retab"}) 
        for cell in self.grid.coord_iter():
            cell_content, x, y = cell
            if self.random.random() < self.density:
                if self.random.random() < self.minority_pc:
                    agent = CovAgent((x, y), self, 1,0,None,random.uniform(0,1),random.uniform(5,15))
                    self.infecte+=1
                else:
                    agent = CovAgent((x, y), self, 0,0,random.uniform(0,1))
                    self.sucep+=1
                if self.random.random() < self.d_pm:
                    agent.port_mask=1
                    
                self.grid.position_agent(agent, (x, y))
                self.schedule.add(agent)

        self.running = True
        self.datacollector.collect(self)

    def step(self):
        self.runs+=1
        self.infecte = 0  
        self.sucep = 0
        self.retab = 0
        self.schedule.step()
        # collect data
        self.datacollector.collect(self)
        if self.infecte == 0:
            self.running = False

In [65]:
def cov_draw(agent):
    portrayal = {"Shape": "circle", "r": 0.6, "Filled": "true"}
    if agent is None:
        return
    if agent.type == 0:
        portrayal["Color"] = ["#FFFF00", "#FFFFBB"]
        portrayal["Layer"] = 0
        if agent.port_mask==0:
            portrayal["r"] = 0.3
    if agent.type == 1 :
        portrayal["Color"] = ["#FF0000", "#FFBBBB"]
        portrayal["Layer"] = 0
        if agent.port_mask==0:
            portrayal["r"] = 0.3
    if  agent.type == -1 :
        portrayal["Color"] = ["#00FF00", "#BBFFBB"]
        portrayal["Layer"] = 0
            
    return portrayal

In [66]:
class Infecte(TextElement):
    def __init__(self):
        pass

    def render(self, model):
        return "agents infectes: " + str(model.infecte)

class Suceptible(TextElement):
    def __init__(self):
        pass

    def render(self, model):
        return "agents suceptibles: " + str(model.sucep)
    
class Retabli(TextElement):
    def __init__(self):
        pass

    def render(self, model):
        return "agents retablis: " + str(model.retab)

In [68]:
canvas_element = CanvasGrid(cov_draw, 20, 20, 500, 500)
infecte_element=Infecte()
sucep_element=Suceptible()
retab_element=Retabli()
all_chart = ChartModule([{"Label": "infecte", "Color": "red"},{"Label": "suceptible", "Color": "yellow"},{"Label": "retabli", "Color": "green"}], data_collector_name='datacollector')
model_params = {
                        "height": 20,
                        "width": 20,
                        "density": UserSettableParameter("slider", "Agent density", 0.75, 0.1, 1.0, 0.1),
                        "minority_pc": UserSettableParameter(
                            "slider", "Fraction des infectes", 0.25, 0.00, 1.0, 0.05),
                        "d_pm": UserSettableParameter("slider", "Fraction des porteurs du mask", 0.25, 0.1, 1.0, 0.05),
                        "confinement": UserSettableParameter('checkbox', 'Confinement', value=False),
}
server = ModularServer(CovModel,
                       [canvas_element, infecte_element,sucep_element,retab_element,all_chart],
                       "Covid 19 Model",
                      model_params
                        )
                    
server.port = 5560# The default
server.launch()

Interface starting at http://127.0.0.1:5560


RuntimeError: This event loop is already running

Socket opened!
{"type":"reset"}
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
